# Privacy preserving machine learning

*Authors: Daniel Meier (Swiss Re Ltd), Michael Mayer (La Mobilière), members of the Data Science Working Group of the Swiss Association of Actuaries, see https://actuarialdatascience.org*, and Juan-Ramón Troncoso-Pastoriza (Tune Insight)

This notebook introduces privacy preserving machine learning methods using synthetic health datasets with risk factors like BMI, blood pressure, age, etc. to predict various health outcomes of individuals over time. The notebook consists of **5 parts**:

1.   Creation of synthetic health datasets
2.   Traditional models: logistic regression/GLM, and Cox regression
3.   Shallow and deep neural networks
4.   Model explainability and risk factor importance
5.   Homomorphic encryption

## Key takeaways

*   Currently (in 2023), there is still limited availability of publicly accessible and sufficiently large and dense health datasets, we therefore work on synthetic datasets
*   Traditional models might be the better choice - depending on the (size of the) dataset
*   Neural networks as an extension of GLMs
*   Understanding some basics of epidemiology: hazard rates/ratios, odds ratios, relative risk
*   How to become comfortable with more complex models: partical dependence plots, individual conditional expectations, permutation importance
*   Introduction into privacy preserving methods, homomorphic encryption
*   Last but not least: insights into general health aspects


# Part 5: Homomorphic encryption
In this last part we showcase the concept of homomorphic encryption by first introducing several helper functions to

* encrypt and decrypt with asymmetric RSA,
* factor larger numbers with the quadratic sieve to break RSA (for small keys),
* encrypt and decrypt with asymmetric ElGamal,
* send/receive messages with asymmetric and symmetric encryption.



In [ ]:
import random, array, time
import numpy as np
from math import exp, log, sqrt

def gcd_ext(a, b):
    # returns gcd, x, y, such that gcd = a*x + b*y
    if a == 0: return b, 0, 1
    d, x, y = gcd_ext(b % a, a)
    return d, y - (b // a) * x, x

def jacobi(a, n):
    # Jacobi symbol (generalized Legendre symbol) for positive, odd n
    a = a % n
    t = 1
    while a != 0:
        while a % 2 == 0:
            a = a // 2
            r = n % 8
            if r == 3 or r == 5: t = -t
        a, n = n, a
        if a % 4 == 3 and n % 4 == 3: t = -t
        a = a % n
    if n == 1: return t
    else: return 0

def is_probable_prime(n, k = 20):
    # Solovay-Strassen probabilistic prime number test
    if n == 2: return True
    if n % 2 == 0 or n == 1: return False
    for j in range(k):
        r = random.randint(2, n-1)
        x = jacobi(r, n) % n
        if x == 0 or pow(r, (n-1)//2, n) != x:
            return False
    return True

def next_probable_prime(n):
    if n % 2 == 0: n += 1
    while not is_probable_prime(n): n += 2
    return n

def text_to_int(text):
    return int.from_bytes(text.encode(), "little")

def int_to_text(n):
    try:
      out = n.to_bytes(int(log(n)/log(256)+1), "little").decode()
    except Exception:
      out = "Invalid integer"
    return out

def rsa_attack(n, e):
    # retrieve private key from public key by factoring n
    factors = MPQS(n).factor()
    p = factors[0]
    q = factors[1]
    d = gcd_ext(e, (p-1)*(q-1))[1] % ((p-1)*(q-1))
    return n, d

The next code block is an implementation of the quadratic sieve to factor integers, see, e.g., https://www.ams.org/notices/199612/pomerance.pdf

In [ ]:
class MPQS:
  def __init__(self, n, max_trial_division=1e6, sieve_window_size=100000, multiplier=10):
    # n, number to be factored
    # max_trial_division, fist perform a trial division up to primes of size max_trial_division
    # sieve_window_size and m are tuning parameters that will impact computation times
    if not n>1 and n%1 == 1: raise ValueError("Invalid n")
    print("Trying to factor {}-digit number {}".format(len(str(n)), n))
    self.sieve_window_size = sieve_window_size
    self.max_trial_division = int(max_trial_division)
    self.multiplier = multiplier
    _, self.n, self.power = self.is_power(n, max_trial_division)
    self.factors, self.n = self.trial_division(self.n, self.max_trial_division)
    self.sqrt_n = sqrt(self.n)
    self.b = int(exp(sqrt(log(self.n)*log(log(self.n)+1e-16))/2))
    self.factor_base = [-1, 2] + [p for p in self.first_primes(self.b)[1:] if jacobi(self.n, p) == 1]
    self.sqrt_p, self.log_p = self.get_roots_logs(self.factor_base, self.n)
    self.exponent_vectors = np.zeros((len(self.factor_base), len(self.factor_base)), dtype=bool)
    self.X_list = [0] * len(self.factor_base)
    self.Y_list = [0] * len(self.factor_base)
    self.row = [-1] * len(self.factor_base)
    self.numbers_found = 0
    self.M = self.multiplier * len(self.factor_base)
    self.A = 0
    self.B = 0
    self.C = 0
    self.D = int(sqrt(sqrt(2*self.n)/self.M))
    self.D_inv = 0
    self.root1 = [0] * len(self.factor_base)
    self.root2 = [0] * len(self.factor_base)

  def factor(self):
    print("MPQS trying to factor {}-digit number {}".format(len(str(self.n)), self.n))
    if is_probable_prime(self.n): return np.repeat(self.factors + [self.n], self.power).tolist()
    if self.n == 1: return np.repeat(self.factors, self.power).tolist()
    factors = self.factor_into_two()
    out = []
    for p in factors:
      if is_probable_prime(p): out.append(p)
      else: out += MPQS(p, self.max_trial_division, self.sieve_window_size, self.multiplier).factor()
    print("\n{}".format(np.repeat(self.factors + out, self.power).tolist()))
    return np.repeat(self.factors + out, self.power).tolist()

  def trial_division(self, n, max_trial_division):
    out = []
    for p in self.first_primes(max_trial_division):
      while n % p == 0:
        n //= p
        out.append(p)
    return out, n

  def is_power(self, n, min_base):
    for r in range(2, int(log(n)/log(min_base) + 2)):
      p = self.int_root(n, r)
      if p != -1: return True, p, r
    return False, n, 1

  def int_root(_, n, r):
    low = int(n**(1/(r+0.1)))
    high = int(n**(1/(r-0.1)))
    while high - low > 2:
      center = high//2 + low//2
      if center**r > n: high = center
      else: low = center
    if high**r == n: return high
    if (high-1)**r == n: return high - 1
    if low**r == n: return low
    return -1

  def first_primes(_, n):
    is_prime = np.ones(n+1, dtype=bool)
    out = []
    i = 2
    while 1:
      while i <= n and not is_prime[i]: i = i + 1
      if i == n+1: return out
      out.append(i)
      idx = np.arange(2*i, n + 1, i)
      is_prime[idx] = False
      i += 1

  def get_roots_logs(self, factor_base, n):
    sqrt_p = [0] * len(factor_base)
    log_p = [0] * len(factor_base)
    for i in range(1, len(factor_base)):
      sqrt_p[i] = self.sqrt_mod_p(n, factor_base[i])
      log_p[i] = log(factor_base[i])
    return sqrt_p, log_p

  def sqrt_mod_p(self, n, p):
    if p == 2: return n % 2
    if p % 4 == 3: return pow(n, (p + 1) // 4, p)
    if p % 8 == 5:
      out = pow(n, (p + 3) // 8, p)
      if pow(out, 2, p) != n % p: out = out * pow(2, (p-1)//4, p) % p
      return out % p
    s = 0
    t = p - 1
    while (t % 2 == 0):
      t = t // 2
      s += 1
    d = 2
    while jacobi(d, p) == 1: d += 1
    nt = pow(n, t, p)
    dt = pow(d, t, p)
    m = 0
    for i in range(0, s-1):
      T = pow(dt, m, p) * nt
      T = pow(T, pow(2, s - 2 - i), p) + 1
      if T == p: m = m + pow(2, i+1)
    return (pow(n, (t+1)//2, p)*pow(dt, m//2, p)) % p

  def set_next_polynomial(self):
    self.D = self.D + (3 - self.D % 4) + 4
    while not is_probable_prime(self.D) or pow(self.n, (self.D-1)//2, self.D) != 1: self.D += 4
    self.A = pow(self.D, 2, self.n)
    H1 = pow(self.n, (self.D + 1)//4, self.D)
    t = gcd_ext(2*H1, self.D)[1] # inverse of 2*H1 mod D
    H2 = t*((self.n-H1*H1)//self.D) % self.D
    self.B = (H1 + H2*self.D) % self.A
    self.C = (self.B*self.B - self.n)//self.A
    self.D_inv = gcd_ext(self.D, self.n)[1] % self.n
    self.root1 = [0] * (len(self.factor_base))
    self.root2 = [0] * (len(self.factor_base))
    for i in range(1, len(self.factor_base)):
      B_p = self.B % self.factor_base[i]
      A_inv_p = gcd_ext(self.A, self.factor_base[i])[1] % self.factor_base[i]
      self.root1[i] = ((self.sqrt_p[i] - B_p) * A_inv_p ) % self.factor_base[i]
      self.root2[i] = ((-self.sqrt_p[i] - B_p) * A_inv_p ) % self.factor_base[i]

  def factor_into_two(self):
    while 1:
      self.set_next_polynomial()
      factors = self.sieve()
      if len(factors) > 1:
        return factors

  def sieve(self):
    for window in range(-self.M//self.sieve_window_size, self.M//self.sieve_window_size):
        sieve = np.zeros(self.sieve_window_size)
        sieve_start = window * self.sieve_window_size
        sieve_end = (window+1) * self.sieve_window_size - 0.5
        qx_start = (self.A*sieve_start+2*self.B)*sieve_start+self.C
        qx_end = (self.A*sieve_end+2*self.B)*sieve_end+self.C
        threshold = log(min(abs(qx_start), abs(qx_end))) - 1
        for f in range(1, len(self.factor_base)):
          r = sieve_start % self.factor_base[f]
          r1 = (self.root1[f] - r) % self.factor_base[f]
          r2 = (self.root2[f] - r) % self.factor_base[f]
          idx = np.concatenate((np.arange(r1, self.sieve_window_size, self.factor_base[f]), np.arange(r2, self.sieve_window_size, self.factor_base[f])))
          sieve[idx] += self.log_p[f]
        for s in np.where(sieve > threshold)[0]:
            exponent_vector = np.zeros(len(self.factor_base), dtype=int)
            x = sieve_start + int(s)
            X = self.A*x + self.B
            qx = (X + self.B)*x + self.C
            X = X*self.D_inv % self.n
            if qx < 0:
              qx = self.n - qx
              exponent_vector[0] = 1
            for f in range(1, len(self.factor_base)):
              r = x % self.factor_base[f]
              if (r != self.root1[f] and r != self.root2[f]): continue
              while qx >= self.factor_base[f] and qx % self.factor_base[f] == 0:
                qx = qx // self.factor_base[f]
                exponent_vector[f] += 1
            if qx == 1: # b-smooth Q(x) has been found
              Y = 1
              idx = np.where(exponent_vector[1:]>=2)[0]+1
              for f in idx:
                Y = (Y * pow(self.factor_base[f], int(exponent_vector[f]) // 2)) % self.n
              exponent_vector[idx] %= 2
              g = gcd_ext(X + Y, self.n)[0] % self.n
              if g > 1 and g < self.n-1: return [g, self.n//g]
              exponent_vector_root = np.zeros(len(self.factor_base), dtype=int)
              for f in np.arange(len(self.factor_base)):
                if exponent_vector[f] % 2 == 0: continue
                if self.row[f] >= 0:
                  X = X*self.X_list[self.row[f]] % self.n
                  Y = Y*self.Y_list[self.row[f]] % self.n
                  exponent_vector += self.exponent_vectors[self.row[f], :]
                  idx = np.where(exponent_vector>=2)[0]
                  exponent_vector_root[idx] += exponent_vector[idx] // 2
                  exponent_vector[idx] %= 2
                  if not np.any(exponent_vector[f:]):
                    for i in np.where(exponent_vector_root[1:]>0)[0]+1:
                      Y *= pow(self.factor_base[i], int(exponent_vector_root[i])) % self.n
                    exponent_vector_root = np.zeros(len(self.factor_base), dtype=int)
                    g = gcd_ext(X + Y, self.n)[0] % self.n
                    if g > 1 and g < self.n-1: return [g, self.n//g]
                else:
                  for i in np.where(exponent_vector_root[1:]>0)[0]+1:
                      Y *= pow(self.factor_base[i], int(exponent_vector_root[i])) % self.n
                  self.row[f] = self.numbers_found
                  self.X_list[self.numbers_found] = X
                  self.Y_list[self.numbers_found] = Y
                  self.exponent_vectors[self.numbers_found, :] = exponent_vector % 2 == 1
                  self.numbers_found += 1
                  print("\rNumber of b-smooth numbers found ({} needed at most): {}".format(len(self.factor_base), self.numbers_found), end="")
                  break
    return [self.n]

The following classes implement asymmetric encryption with RSA, ElGamal, and symmetric encryption based on BBS (Blum-Blum-Shub, see https://shub.ccny.cuny.edu/articles/1986-A_simple_unpredictable_pseudo-random_number_generator.pdf).

In [ ]:
class RSA:
  def __init__(self, p=0, q=0, e=0):
    size = 100000000000000000
    if p==0: p = next_probable_prime(random.randint(1*size, 2*size))
    if q==0: q = next_probable_prime(random.randint(3*size, 4*size))
    if e==0:
        self.e = next_probable_prime(random.randint(0, (p-1)*(q-1)))
        while gcd_ext(self.e, (p-1)*(q-1))[0] != 1: self.e = (self.e + 1) % ((p-1)*(q-1))
    else: self.e = e
    self.n = p*q
    if gcd_ext(self.e, (p-1)*(q-1))[0] != 1: raise ValueError("e and (p-1)*(q-1) need to be coprime")
    self._d = gcd_ext(self.e, (p-1)*(q-1))[1] % ((p-1)*(q-1))

  def get_public_key(self):
    return self.n, self.e

  def get_private_key(self):
    return self.n, self._d

  def encrypt(self, m, n=0, e=0):
    if n==0: n = self.n
    if e==0: e = self.e
    if m>n: raise ValueError("m too large, n too small")
    return pow(m, e, n)

  def decrypt(self, c, n=0, d=0):
    if n==0: n = self.n
    if d==0: d = self._d
    return pow(c, d, n)

class ElGamal:
  def __init__(self, p=0, g=0):
    size = 100000000000000000
    if p==0:
      self.p = 1
      while not is_probable_prime(self.p):
        p1, p2 = next_probable_prime(random.randint(1*size, 2*size)), next_probable_prime(random.randint(3*size, 4*size))
        self.p = 2*p1*p2 + 1
      self.g = random.randint(2, self.p-2)
      while pow(self.g, 2*p1, self.p) == 1 or pow(self.g, 2*p2, self.p) == 1 or pow(self.g, p1*p2, self.p) == 1:
        self.g = random.randint(2, self.p-2)
    else:
      self.p = p
      self.g = g
    self._x = random.randint(1, self.p-2)
    self.h = pow(self.g, self._x, self.p)
    self.nonce_list = []

  def get_public_key(self):
    return self.p, self.g, self.h

  def get_private_key(self):
    return self._x

  def encrypt(self, m, p=0, g=0, h=0):
    if p==0: p = self.p
    if g==0: g = self.g
    if h==0: h = self.h
    nonce = random.randint(1, p-2)
    while nonce in self.nonce_list:
      nonce = random.randint(1, p-2)
    self.nonce_list.append(nonce)
    return m*pow(h, nonce, p) % p, pow(g, nonce, p)

  def decrypt(self, c, d):
    return c*gcd_ext(pow(d, self._x, self.p), self.p)[1] % self.p

class BBS:
  def __init__(self, n=0, seed=0):
    # create a pseudo-random sequence of period length 4*p1*p2*q1*q2, Blum-Blum-Shub
    size = 10000000
    self.skip = 1000
    if n==0:
        p, q = 0, 0
        while not(is_probable_prime(p) and is_probable_prime(q)):
            p1, p2 = random.randint(1*size, 2*size), random.randint(3*size, 4*size)
            q1, q2 = random.randint(5*size, 6*size), random.randint(7*size, 8*size)
            p, q = 2*p1*p2+1, 2*q1*q2+1
            n = p*q
    self.n = n
    self.number_bytes = int(log(self.n)/log(256)+1)
    self.nonce_list = []
    if seed == 0: self._seed = random.randint(2,self.n)
    else: self._seed = seed

  def next(self, length, nonce):
    current = (nonce + self._seed) % self.n
    out = bytearray(length)
    for i in range(self.skip):
        current = pow(current, 2, self.n)
    for i in range(length * 8):
        current = pow(current, 2, self.n)
        out[i//8] += (sum(current.to_bytes(self.number_bytes, "little")) % 2) << (i%8)
    return out

  def encrypt(self, message, nonce):
    if nonce in self.nonce_list: raise ValueError("nonce was already used")
    self.nonce_list.append(nonce)
    bytes = message.encode()
    return bytearray([x^y for x, y in zip(bytes, self.next(len(bytes), nonce))])

  def decrypt(self, encrypted_message, nonce):
    return bytearray([x^y for x, y in zip(encrypted_message, self.next(len(encrypted_message), nonce))]).decode()

class Person:
  def __init__(self, name, secret_message):
    self.name = name
    self._secret_message = secret_message
    self._rsa = RSA()
    self._bbs = BBS()
    self._elg = ElGamal()

  def get_rsa_public_key(self):
    return self._rsa.get_public_key()

  def get_rsa_encrypted_seed(self, person):
    return self._bbs.n, RSA().encrypt(self._bbs._seed, *person.get_rsa_public_key())

  def get_elg_public_key(self):
    return self._elg.get_public_key()

  def get_elg_encrypted_seed(self, person):
    c, d = ElGamal().encrypt(self._bbs._seed, *person.get_elg_public_key())
    return self._bbs.n, c, d

  def send_encrypted_message(self):
    nonce = int(time.time()*1000)
    encrypted_message = self._bbs.encrypt(self._secret_message, nonce)
    return encrypted_message, nonce

  def receive_encrypted_message_rsa(self, person):
    bbs_n, bbs_encrypted_seed = person.get_rsa_encrypted_seed(self)
    bbs_seed = self._rsa.decrypt(bbs_encrypted_seed)
    person_bbs = BBS(bbs_n, bbs_seed)
    encrypted_message, nonce = person.send_encrypted_message()
    message = person_bbs.decrypt(encrypted_message, nonce)
    print('{} received the secret message "{}" from {}'.format(self.name, message, person.name))

  def receive_encrypted_message_elg(self, person):
    bbs_n, bbs_encrypted_seed, d = person.get_elg_encrypted_seed(self)
    bbs_seed = self._elg.decrypt(bbs_encrypted_seed, d)
    person_bbs = BBS(bbs_n, bbs_seed)
    encrypted_message, nonce = person.send_encrypted_message()
    message = person_bbs.decrypt(encrypted_message, nonce)
    print('{} received the secret message "{}" from {}'.format(self.name, message, person.name))

This section demonstrates the homomorphic capacites of RSA as a didactic exercise. As mentioned in the companion paper, RSA does not have semantic security, so it cannot be used securely for homomorphic processing in practical applications.

In [ ]:
rsa = RSA(next_probable_prime(100000000000000000), next_probable_prime(1100000000000000000), 17)
cipher = rsa.encrypt(text_to_int("Data Science"))
print("Decrypting message with private key: {}".format(int_to_text(rsa.decrypt(cipher))))

print("\nAttacking RSA by factoring public key via quadratic sieve:")
n, e = rsa.get_public_key()
factors = MPQS(n).factor()
p, q = factors[0], factors[1]
d = gcd_ext(e, (p-1)*(q-1))[1] % ((p-1)*(q-1))
print(int_to_text(RSA().decrypt(cipher, n, d)))

print("\nHomomorphic RSA encryption example: multiplication <-> multiplication")
a = random.randint(0,1000000)
b = random.randint(0,1000000)
print(rsa.encrypt(a % rsa.n)*rsa.encrypt(b % rsa.n) % rsa.n)
print(rsa.encrypt(a*b % rsa.n) % rsa.n)

Decrypting message with private key: Data Science

Attacking RSA by factoring public key via quadratic sieve:
Trying to factor 36-digit number 110000000000000009600000000000000189
MPQS trying to factor 36-digit number 110000000000000009600000000000000189
Number of b-smooth numbers found (749 needed at most): 707
[1100000000000000063, 100000000000000003]
Data Science

Homomorphic RSA encryption example: multiplication <-> multiplication
82220119621987860348718849059267407
82220119621987860348718849059267407


In [ ]:
alice = Person("Alice", "Alice's secret message")
bob = Person("Bob", "Bob's secret message")
bob.receive_encrypted_message_rsa(alice)
bob.receive_encrypted_message_elg(alice)

Bob received the secret message "Alice's secret message" from Alice
Bob received the secret message "Alice's secret message" from Alice


## Example of basic encrypted processing using Tune Insight's client-side `cryptolib/hefloat`

This notebook exemplifies a fully encrypted workflow where the client has a database of individual data records and wants to apply a mortality model (logistic regression) to predict the mortality risk for the individuals represented in each data record.
For this, we use a light version of Tune Insight's public `cryptolib/hefloat` library, which provides a simplified interface to abstract cryptographic concepts and enable quick and simple manipulation of encrypted records. These are the followed steps:
- Parameterization of the circuit and cryptosystem instantiation
- Key generation
- Polynomial approximation of the model activation functions
- Batch (packed) encryption of all input records and model coefficients
- Homomorphic evaluation of the model prediction under encryption
- Decryption of the prediction results
- Accuracy comparison with clear-text non-encrypted equivalent processing

If you havea any questions or find any issues with the library, do not hesitate to contact us as [contact@tuneinsight.com](mailto:contact@tuneinsight.com).

### Parameterization and cryptosystem instantiation
The `hefloat` library works with approximate arithmetic and uses a cryptosystem adapted to machine learning operations (CKKS).

First, we select the required precision for the homomorphic computations (equivalent to the input scale $\Delta$) and the depth of the circuit (maximum number of consecutive products before the results are decrypted). The library automatically selects secure parameters as a function of the required circuit depth and precision.

In [ ]:
from tuneinsight.cryptolib.hefloat import hefloat

# Parameterization: scale/precision and circuit depth
log_scale = 45                                  # Fixed-point arithmetic floating point scaling factor in bits (log2(Delta))
levels = 7                                      # Circuit depth
log_qi = [log_scale+5] + levels*[log_scale]     # 5 additional bits for the lowest level, to account for plaintext growth
log_pi = [log_scale+5]                          # Auxiliary module used for relinearization (usually, at least of the same size as the lowest level q0)


# In order to generate an instance of the cryptosystem, the RLWE ring degree is automatically chosen to ensure at least 128-bit of security
# A context stores the scheme cryptographic parameters and a key generator
context = hefloat.new_context(log_qi = log_qi, log_default_scale= log_scale, log_pi = log_pi)

#Print some information about the cryptographic parameters
print(f'Log2 N: {context.parameters.log_n()}')
print(f'Log2 Moduli Chain: Q{log_qi} + P{log_pi}')
print(f'Log2 QP: {context.parameters.log_q() + context.parameters.log_p()}')
print(f'Log2 Slots: {context.parameters.log_slots()}')
print(f'Available Depth: {levels}')

### Key Generation
The owner of the data generates a public-private key pair.

First, the secret key must remain at the data owner, as it is the key that can decrypt any ciphertext.

The evaluator object holds the public keys (inlcuding the relinearization key), in order to enable the encrypted execution of homomorphic circuits, but the evaluator alone cannot perform decryption operations, as the secret key is not stored in it.

Therefore, the client can send the evaluator and its public keys to the server where the encrypted computation must be carried out, with the guarantee that the server will not be able to access any clear-text data.

In [ ]:
# Generate a fresh secret key
sk = context.new_secret_key()

# Instantiate an evaluator with a relinearization key
# The relinearization key is a public-evaluation key required to ensure ciphertext x ciphertext compactness
# The resulting evaluator object contains only public information and can be freely shared
evaluator = context.new_evaluator(context.new_relinearization_key(sk))

## Polynomial approximation of activation functions

The cryptosystem naturally supports homomorphic polynomial operations, but non-polynomial operations have to be approximated.

A Chebyshev's polynomial approximation is numerically stable and enables a low approximation error (close to the minimax approximation polynomial) across the defined input interval. In this case (logistic regression), the to-be-approximated function is a Sigmoid. The approximation interval is chosen as a function of the domain spanned by the inputs to the Sigmoid function (after the scalar product between the input vector and the regression coefficient vector). The degree of the approximation polynomial ($2^8-1=63$ in our case) can be chosen to fit the available depth with which the cryptosystem has been parameterized.

### Synthetic data generation
For this example, we generate a table of uniformly distributed random data that represents $l=2^{13}=8192$ records with $k=200$ features. The model coefficients (regression coefficients $\beta_i$, including the bias or intercept coefficient $\beta_0$) are also randomly chosen. In a real scenario, these coefficients will be obtained from the model provider or from a prior training process. The latter can be run on local data or on third party or distributed data, in which case it can also leverage encrypted computation (see the companion notebook using the full Tune Insight SDK for an example of encrypted distributed training with dataset $D_2$).

In [ ]:
import numpy.polynomial.chebyshev as chebyshev
import numpy as np

# Expected interval of the encrypted values after the scalar product
a = -12
b = 12

# Interpolates the Sigmoid in the interval [-12, 12] and returns the coefficients
# for the Chebyshev approximation polynomial in the Chebyshev basis
coeffs = chebyshev.chebinterpolate(lambda x: 1/(1+np.exp(-((b-a)/2 * x + (b+a)/2))), 63)


## Synthetic data generation:
# Number of samples to process in parallel (available plaintext slots that one encryption can hold)
batch_size = context.slots()

# Number of features (k=200)
features = 200

# Generate random data in [-0.5, 0.5]. This is the matrix A'
data = np.random.rand(batch_size, features)-0.5

# Generate random regression weights in [0, 1]. These represent beta_i, i=1,...,k
weights = np.random.rand(features, 1)

# Generate random bias (intercept coefficient) in [0, 1]. This represents beta_0
bias = np.random.rand(1)

## Packed batched encryption of all input records and model coefficients

In order to leverage the inherent parallelization of the cryptosystem (SIMD - Single Instruction Multiple Data) offered by the underlying polynomial/vector arithmetic, we encrypt input values packed into a polynomial/vector representation.

In this case, we exemplify vertical packing (see Figure 11 of the tutorial paper), where each encryption contains a vector of values from multiple data records (one column of the data matrix $A'$). The input values are encoded in the slots (batched), in order to enable component-wise homomorphic operations.

In [ ]:
# This optional parameter defines whether the input vectors will be encoded in the coefficients domain (if batched=False)
# or in the slots domain (if batched=True). The latter is the default behavior, and it enables component-wise homomorphic operations
# (additions and products)
batched = True

# The encrypt function can receive a two-dimensional matrix as input, in which case it encrypts each row of the input matrix in one ciphertext.
# Therefore, we transpose the input A', in order to encrypt each column of A' in one ciphertext.
# We need to explicitly make a copy to ensure a correct memory
# alignment when passing C pointers of arrays to the Go wrapper.
# The function returns an object that stores a vector of ciphertexts.
encrypted_data = context.encrypt(data.transpose().copy(), sk, batched)

# As for the regression coefficients, we encrypt each of the weights replicated in all slots of the corresponding ciphertext.
# For this, we apply repetition coding (with tile) and pass the resulting matrix as input to the encrypt function, so that each row is encrypted in a separate ciphertext.
# The result is an object that stores a vector of ciphertexts, each containing one regression coefficient replicated in all its slots.
encrypted_weights = context.encrypt(np.tile(weights, (1, batch_size))* 2/(b-a), sk, batched)

# The intercept coefficient or bias is also encrypted in its own ciphertext, with the same repetition coding as all the other regression coefficients
encrypted_bias = context.encrypt(np.tile(bias, (1, batch_size))* 2/(b-a) + (-a-b)/(b-a), sk, batched)

## Homomorphic evaluation of the model prediction under encryption

The prediction for the logistic regression involves, for each data record, the computation of

$y_i=\mu(\beta_0+\sum_{j=1}^k a_{i,j}\beta_j)$.

This can be broken down into:
- A scalar product between the data vector and the regression coefficients vector
- The addition of the intercept (bias) coefficient $\beta_0$
- The computation of the Sigmoid function on the result of the previous addition

Thanks to the used vertical packing, these three subsequent operations can be executed in parallel over all the data records as if we were manipulating a single record (SIMD).

### Scalar product and addition of bias

The scalar product is evaluated using homomorphic products and additions. The `scalar_product` function takes two vectors of ciphertexts, and computes the scalar product between them, which is the homomorphic equivalent of performing $l=8192$ parallel scalar products between the $l$ $200$-length data vectors with the regression coefficient vector. The addition of the intercept is also a parallel operation

In [ ]:
# Encrypted evaluation of data @ weights computed as np.sum(data.transpose() * np.tile(bias, (1, batch_size)), axis=0)
# This is faster, but equivalent, to doing evaluator.sum(evaluator.mul(encrypted_data, encrypted_weights), axis=0)
encrypted_scalar_product = evaluator.scalar_product(encrypted_data, encrypted_weights)

# Encrypted evaluation of data @ weights + bias
encrypted_scalar_product_plus_bias = evaluator.add(encrypted_bias, encrypted_scalar_product)

### Sigmoid evaluation

The Sigmoid is evaluated through the approximation polynomial calculated above. The function `polynomial` takes the coefficientes of the Chebyshev decomposition of a given polynomial and evaluates this polynomial on the input encryption(s). This is also a SIMD operation, where the polynomial evaluation is homomorphically applied component-wise to the all the slots in the ciphertext, so the result is indeed the vector of predictions for all the input data records, one prediction in each of the slots of the resulting ciphertext.

In [ ]:
# Encrypted evaluation of sigmoid(data @ weights + bias)
encrypted_prediction = evaluator.polynomial(encrypted_scalar_product_plus_bias, coeffs=coeffs, basis="Chebyshev")

## Decryption of the prediction results

The decryption requires the secret key to succeed, and produces as a result the vector of $l=8192$ predictions.

In [ ]:
# Decrypt the values
prediction = context.decrypt(encrypted_prediction, sk)[:, :batch_size]

## Accuracy comparison with clear-text non-encrypted equivalent processing

In order to measure the precision preserved by the whole encrypted process, we compute the same logistic regression on the cleartext data, and compute the log of the average square error, which gives us the preserved bits of precision, which roughly equal 32 bits in this case. The scale of the inputs can be modified when parameterizing the cryptosystem in the first step, in order to adapt the precision to the required target.

In [ ]:
from math import log
# Finally, we evaluate the plaintext circuit
clear_target = 1/(np.exp(-(data @ weights + bias))+1)

# And compare with the decrypted result
print(f'Obtained: {prediction}')
print(f'Clear_tg: {clear_target.transpose()}')
print(f'Precision as -log2(avg_l2(obtained-clear_tg))): {-log(np.sqrt(np.sum((prediction-clear_target.transpose())**2))/batch_size, 2)}')

##Contact

For more information on the encrypted statistical and machine learning capabilities of the full secure federated platform, do not hesitate to contact as at [contact@tuneinsight.com](mailto:contact@tuneinsight.com).